In [1]:
import pandas as pd
import os
import numpy as np

In [31]:
df = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/train_data/train.csv', header=None)

In [32]:
df_test = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/test_data/test.csv', header=None)

In [33]:
df_test = df.rename({0: 'label', 1: 'text'}, axis = 1)

In [34]:
df = df.rename({0: 'label', 1: 'text'}, axis = 1)

In [35]:
df.head()

,label,text
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [61]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(data, percentage_of_sentences=None):
    
    train_sentences = data['text']
    y_train = data['label']
    
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
    X_train = [text_to_word_sequence(_) for _ in train_sentences]
    
  
    return X_train, y_train



In [62]:
X_train, y_train = load_data(df, percentage_of_sentences=10)

In [63]:
X_test, y_test = load_data(df_test, percentage_of_sentences=10)

In [79]:
y_test_adj = y_test -1

In [64]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
    
# We apply the tokenization to the train and test set
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)


X_train_pad = pad_sequences(X_train_token, maxlen=50, dtype='float32', padding="post")
X_test_pad = pad_sequences(X_test_token, maxlen=50, dtype='float32', padding="post")

vocab_size = len(tokenizer.word_index)

In [73]:
from tensorflow.keras import Sequential, layers

def init_cnn_model(vocab_size):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size + 1, output_dim=20, mask_zero=True, input_length=50))
    model.add(layers.Conv1D(32, 3))
    model.add(layers.Flatten())
    model.add(layers.Dense(50, activation="relu"))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_cnn = init_cnn_model(vocab_size)

In [74]:
model_cnn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 50, 20)            1378040   
                                                                 
 conv1d_9 (Conv1D)           (None, 48, 32)            1952      
                                                                 
 flatten_9 (Flatten)         (None, 1536)              0         
                                                                 
 dense_18 (Dense)            (None, 50)                76850     
                                                                 
 dense_19 (Dense)            (None, 1)                 51        
                                                                 
Total params: 1,456,893
Trainable params: 1,456,893
Non-trainable params: 0
_________________________________________________________________


In [75]:
y_train_adj = y_train -1

In [76]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad, y_train_adj, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


Epoch 1/20
1225/1225 [==============================] - 14s 5ms/step - loss: 0.3589 - accuracy: 0.8351 - val_loss: 0.2824 - val_accuracy: 0.8826
Epoch 2/20
1225/1225 [==============================] - 7s 5ms/step - loss: 0.1924 - accuracy: 0.9232 - val_loss: 0.3229 - val_accuracy: 0.8700
Epoch 3/20
1225/1225 [==============================] - 7s 5ms/step - loss: 0.0789 - accuracy: 0.9707 - val_loss: 0.4508 - val_accuracy: 0.8575
Epoch 4/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.0257 - accuracy: 0.9915 - val_loss: 0.6846 - val_accuracy: 0.8537
Epoch 5/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.0130 - accuracy: 0.9954 - val_loss: 0.9301 - val_accuracy: 0.8450
Epoch 6/20
1225/1225 [==============================] - 6s 5ms/step - loss: 0.0121 - accuracy: 0.9958 - val_loss: 0.9433 - val_accuracy: 0.8487


In [80]:
res = model_cnn.evaluate(X_test_pad, y_test_adj)

1750/1750 [==============================] - 1s 505us/step - loss: 0.2230 - accuracy: 0.9138


In [81]:
!git add .

In [82]:
!git commit -m "trained CNN"

[master fef80bf] trained CNN
 8 files changed, 5904 insertions(+), 3200 deletions(-)
 create mode 100644 .ipynb_checkpoints/review_data_cleaning_baseline-checkpoint.ipynb
 rename .ipynb_checkpoints/{Untitled-checkpoint.ipynb => sequential_model-checkpoint.ipynb} (100%)
 create mode 100644 .ipynb_checkpoints/trained_cnn-checkpoint.ipynb
 delete mode 100644 Untitled.ipynb
 delete mode 100644 review_data_cleaning.ipynb
 create mode 100644 review_data_cleaning_baseline.ipynb
 create mode 100644 sequential_model.ipynb
 create mode 100644 trained_cnn.ipynb


In [ ]:
!git checkout -b 